In [1]:
import folium
import branca.colormap as cm
import networkx as nx
import geojson as gjs
from shapely import geometry, affinity

from clust_tools import clustering as cl
from clust_tools import spatial as sp

In [2]:
# Reading graphs

metro = nx.read_gml("data/graphs/metro.gml")
bus = nx.Graph(nx.read_gml("data/graphs/bus.gml").to_undirected())
tram = nx.Graph(nx.read_gml("data/graphs/tram.gml").to_undirected())

In [3]:
# Computing some measures

print(nx.density(metro))
print(nx.number_of_nodes(metro))
print(nx.number_of_edges(metro))

0.01958670260557053
106
109


In [4]:
print(nx.density(bus))
print(nx.number_of_nodes(bus))
print(nx.number_of_edges(bus))

0.0009721370160382383
2110
2163


In [5]:
print(nx.density(tram))
print(nx.number_of_nodes(tram))
print(nx.number_of_edges(tram))

0.004424514794782055
449
445


# Metro network
Computing the neighborhood graph of the metro graph and plotting it together with the original metro network

In [6]:
# Computing the neighborhood graph of the metro graph

metro_nodes = list(metro.nodes(data = True))
# Clustering stops with a radius of 2km
metro_regions, _ = cl.cluster_stops(metro_nodes, 2)
metro_points = [x["centroid"] for _, x in metro_regions.items()]

# Obtaining neighborhood structure where a connection between two cells is given if: 
# 1. cells are adjacent
# 2. the distance between two cells is at most 5km
metro_region_neighbors = cl.get_neighbor_list(metro_points, radius=5)
# Getting neighbor graph
metro_neighbor_graph = nx.Graph(metro_region_neighbors)

In [35]:
# Plotting the metro graph

dark_blue = "#003366"
m = folium.Map(location=[45.46427, 9.18951], zoom_start = 11.4)

metro_color = {
    "M1": "red",
    "M2": "green",
    "M3": "yellow",
    "M5": "purple"
}

for e in metro.to_directed().edges:
    stop_coord_1 = [float(i) for i in metro.nodes[e[0]]["coord"].split(",")]
    stop_coord_2 = [float(i) for i in metro.nodes[e[1]]["coord"].split(",")]

    metro_edge_coords = [stop_coord_1, stop_coord_2]
    folium.PolyLine(metro_edge_coords, color = dark_blue, weight = 1.3).add_to(m)
    
for stop in metro_nodes:
    stop_coord = [float(i) for i in stop[1]["coord"].split(",")]
    routes = stop[1]["routes"].split(",")
    
    folium.Circle(
        radius = 150,
        location = stop_coord,
        popup = "<b>" + stop[0] + "</b>" + "<br>" + ", ".join(routes),
        color = metro_color[routes[0]],
        fill = True,
        fill_color= metro_color[routes[0]],
        fill_opacity = 0.5
    ).add_to(m)
    
m

In [8]:
# Plotting the metro neighborhood graph

m = folium.Map(location=[45.46427, 9.18951], zoom_start = 11.4)

for e in metro_neighbor_graph.to_directed().edges:
    metro_edge_coords = [metro_regions[e[0]]["centroid"], metro_regions[e[1]]["centroid"]]
    folium.PolyLine(metro_edge_coords, color = dark_blue, weight = 1.3).add_to(m)

for n in metro_neighbor_graph:
    metro_neighbor_graph.nodes[n]["centroid"] = metro_regions[n]["centroid"]
    metro_neighbor_graph.nodes[n]["members"] = metro_regions[n]["members"]
    
    folium.Circle(
        radius = 150,
        location = metro_neighbor_graph.nodes[n]["centroid"],
        tooltip = "<b>Region: " + str(n) + "</b>" + "<br>" + ",<br> ".join(metro_neighbor_graph.nodes[n]["members"]),
        color = dark_blue,
        fill = True,
        fill_color=dark_blue,
        fill_opacity = 0.9
    ).add_to(m)
    
linear = cm.linear.Set1_08.scale(0, len(metro_regions)).to_step(len(metro_regions))
linear.caption = "Regions"
m.add_child(linear)

for key, val in metro_regions.items():
    for c, elem in enumerate(val["members"]):

        folium.Circle(
            radius = 100,
            location = val["coords"][c],
            tooltip = "Region: " + str(key) + "<br>" + elem,
            color = linear(key),
            fill = True,
            fill_color = linear(key),
            fill_opacity=0.3,
        ).add_to(m)
m

# Public transport network
Computing the neighborhood graph, Voronoi diagram and cell-to-cell flow graph of the whole transport net and plotting them

In [9]:
# Merging networks, computing neighbor graph

# Bus and tram share some stops, some processing is needed before uniting the nets. In the composition, bus attributes take the precedence
bus_tram = nx.compose(tram, bus)

for node in tram.nodes(data = True):
    if node[1]["routes"] != bus_tram.node[node[0]]["routes"]:
        bus_tram.node[node[0]]["routes"] += "," + (node[1]["routes"])
        
net = nx.union(metro, bus_tram)
stops = list(net.nodes(data = True))

# Clustering stops
regions, stops_mapping = cl.cluster_stops(stops, 2)

points = [x["centroid"] for _, x in regions.items()]

# Computing neighbor structure
region_neighbors = cl.get_neighbor_list(points, radius=5)
# Getting neighbor graph
neighbor_graph = nx.Graph(region_neighbors)

## Neighborhood graph

In [10]:
# Plotting the net's neighborhood graph (it's a Delaunay triangulation!)

m = folium.Map(location=[45.46427, 9.18951])
m.fit_bounds([[min(points)[0], min(points)[1]], [max(points)[0], max(points)[1]]])

for e in neighbor_graph.to_directed().edges:
    edge_coords = [regions[e[0]]["centroid"], regions[e[1]]["centroid"]]
    folium.PolyLine(edge_coords, color = dark_blue, weight = 1.3).add_to(m)

for n in neighbor_graph:
    neighbor_graph.nodes[n]["centroid"] = regions[n]["centroid"]
    neighbor_graph.nodes[n]["members"] = regions[n]["members"]
    
    folium.Circle(
        radius = 150,
        location = neighbor_graph.nodes[n]["centroid"],
        popup = "<b>" + str(n) + "</b>" + "<br>" + ", ".join(neighbor_graph.nodes[n]["members"]),
        color = dark_blue,
        fill = True,
        fill_color = dark_blue,
        fill_opacity = 0.9
    ).add_to(m)
    
m

## Voronoi diagram

In [11]:
# Computing the net's Voronoi diagram (it's the complementary graph to the Delaunay traingulation)

# Inverting latitude and longitude because GeoJSON
i_points = [p[::-1] for p in points]

# Binding Voronoi's infinite cells
# Method 3: Upscaled convex hull + poly intersection
hull = sp.convex_hull(i_points)
hull_poly = [hull.points[vertex] for vertex in hull.vertices]
boundaries = geometry.Polygon(hull_poly)
# Upscaling the convex hull to have decent regions on the border centroids
boundaries = affinity.scale(boundaries, xfact=1.1, yfact=1.1)

vor = sp.voronoi_finite(i_points)
vor_cells = [vor["vertices"][region].tolist() for region in vor["regions"]]

# For every polygon generated by Voronoi, if the polygon overlaps the convex hull it is intersected to bind it
# mp is a list of polygons representing the binded Voronoi diagram
mp = []
for elem in vor_cells:
    poly = geometry.Polygon(elem)
    if poly.overlaps(boundaries):
        poly = poly.intersection(boundaries) 
    mp.append(poly)


In [12]:
# Plotting the Voronoi diagram

feature_coll = []
for i, pol in enumerate(mp):
    feature_coll.append(gjs.Feature(geometry = pol, id=i))
feature_coll = gjs.FeatureCollection(feature_coll)

m = folium.Map(location=[45.46427, 9.18951])
m.fit_bounds([[min(points)[0], min(points)[1]], [max(points)[0], max(points)[1]]])

linear = cm.linear.Accent_08.scale(0, len(mp)).to_step(len(mp))
linear.caption = "Region"
m.add_child(linear)

folium.GeoJson(feature_coll,
              style_function=lambda feature: {
                'fillColor': linear(feature['id']),
                'color': 'white',
                'weight': 2,
                'dashArray': '10,10',
                'fillOpacity': 0.6,
                }).add_to(m)

for n in neighbor_graph:
    neighbor_graph.nodes[n]["centroid"] = regions[n]["centroid"]
    neighbor_graph.nodes[n]["members"] = regions[n]["members"]
    
    folium.Circle(
        radius = 150,
        location = neighbor_graph.nodes[n]["centroid"],
        popup = "<b>Region: " + str(n) + "</b>",
        color = dark_blue,
        fill = True,
        fill_color= dark_blue,
        fill_opacity = 0.9
    ).add_to(m)

m

## Cell-to-cell graph

In [13]:
# Computing the c2c graph
c2c_graph = cl.c2c_flow_graph(net, stops_mapping)

In [14]:
# Plotting the c2c graph
m = folium.Map(location=[45.46427, 9.18951])
m.fit_bounds([[min(points)[0], min(points)[1]], [max(points)[0], max(points)[1]]])

max_weight = max(dict(c2c_graph.edges).items(), key=lambda x: x[1]['weight'])[1]["weight"]

linear = cm.linear.RdYlGn_11
linear.caption = "Weight"
m.add_child(linear)

for e in c2c_graph.to_directed().edges(data = True):
    weight = c2c_graph[e[0]][e[1]]["weight"]
    edge_coords = [regions[e[0]]["centroid"], regions[e[1]]["centroid"]]
    folium.PolyLine(
        edge_coords,
        color = linear(weight/max_weight),
        weight = 3 * weight/max_weight * 5,
        tooltip = "Weight:<b> " + str(weight) + "</b>" + "<br>"
       ).add_to(m)

for n in c2c_graph:
    
    folium.Circle(
        radius = 150,
        location = regions[n]["centroid"],
        tooltip = "Region:<b> " + str(n) + "</b>" + "<br>",
        color = dark_blue,
        fill = True,
        fill_color=dark_blue,
        fill_opacity = 0.9
    ).add_to(m)

m

## Saving graphs

In [15]:
# Saving graphs
nx.write_gml(neighbor_graph, "data/graphs/net_neighbor.gml")
nx.write_gml(c2c_graph, "data/graphs/net_c2c.gml")
